In [2]:
#install dependencies
!pip install -U transformers==3.0.0
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
!python -m nltk.downloader punkt
from nltk.tokenize import sent_tokenize
!pip install textblob
from textblob import TextBlob
!git clone https://github.com/patil-suraj/question_generation.git
%cd question_generation
!pip install pipelines
from pipelines import pipeline
!pip install sentencepiece

  Using cached transformers-3.0.0-py3-none-any.whl (754 kB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached tokenizers-0.8.0rc4-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.1
    Uninstalling transformers-4.26.1:
      Successfully uninstalled transformers-4.26.1
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
/opt/conda/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARAL

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = TFGPT2LMHeadModel.from_pretrained('gpt2-large')

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/764 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.10G [00:00<?, ?B/s]

In [4]:
#function for the very first prompt
def generate_story(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors='tf')
    output = model.generate(input_ids, max_length=150, do_sample=True, top_p=0.92, top_k=50)
    story = tokenizer.decode(output[0], skip_special_tokens=True)
    return story

#function for all the subsequent prompts, dynamically increasing size
def generate_story_larger(prompt,length):
    input_ids = tokenizer.encode(prompt, return_tensors='tf')
    output = model.generate(input_ids, max_length=length*2, do_sample=True, top_p=0.92, top_k=50)
    story = tokenizer.decode(output[0], skip_special_tokens=True)
    return story


In [10]:
#take a writing prompt and send to GPT, generate Paragraph1 which will be shown to user
prompt = "You are the last human. To cope with loneliness you created androids, who later created more of themselves and started to worship you as a God. You have grown old and know your time's coming."
story = generate_story(prompt)

#cut off everything after the last period
output_string1 = story[:story.rfind('.')]
output_string1=output_string1+"."
print(output_string1)

You are the last human. To cope with loneliness you created androids, who later created more of themselves and started to worship you as a God. You have grown old and know your time's coming. You have a plan. The plan is to survive. But there is only one way to do it: you need someone who can help you find them.

You need a partner that will be able to help you find your way. A partner that can help you find you again. A partner that will love you. Someone who can help you, who will always love you, who will always take care of you. Someone who will accept you for who you are, even if you cannot love someone else.


In [6]:
#Sentiment analysis on the Paragraph1, this score will be used as baseline to judge next prompt
# Create a TextBlob object
blob = TextBlob(output_string1)

# Get the sentiment polarity score
initialscore = blob.sentiment.polarity

# Print the score
print("Initial Sentiment score:", initialscore)

Initial Sentiment score: 0.05281385281385281


In [ ]:
#storing number of words in Paragraph1
count = len(output_string1.split())

#generate two paragraphs from Paragraph1, both will be hidden from user. Both will be used to generate user choice options later
newscore = 1000
while(newscore>=(initialscore)): #keep generating until sentiment/drama score is lower than paragraph1's score
    output_string_temp1 = generate_story_larger(output_string1,count)
    blob = TextBlob(output_string_temp1)
    newscore = blob.sentiment.polarity
    
# Print the temp paragraph 1 sentiment score
print("Temp paragraph 1 Sentiment score:", newscore)   

output_string_temp1 = output_string_temp1[:output_string_temp1.rfind('.')]
output_string_temp1=output_string_temp1+"."
print(output_string_temp1)

print("-------------------")

newscore = 1000
while(newscore>=(initialscore)):
    output_string_temp2 = generate_story_larger(output_string1,count)
    blob = TextBlob(output_string_temp2)
    newscore = blob.sentiment.polarity
    
# Print the temp paragraph 2 sentiment score
print("Temp paragraph 2 Sentiment score:", newscore)

output_string_temp2 = output_string_temp2[:output_string_temp2.rfind('.')]
output_string_temp2=output_string_temp2+"."
print(output_string_temp2)

In [8]:
nlp = pipeline("question-generation")

#number of sentences in Paragraph1,so we know when the newly generated temp paragraphs start
num_sentences_first = len(sent_tokenize(output_string1))



Downloading:   0%|          | 0.00/627 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
#convert to sentences from temp paragraph1, and get the first sentence, then use that first sentence to create a Q1 and A1 which will be used as a choice for user
sentences = sent_tokenize(output_string_temp1)
first_sentence = sentences[num_sentences_first]
option1 = nlp(first_sentence)[0]['question']
answer1 = nlp(first_sentence)[0]['answer']

In [ ]:
#changing up the nlp pipeline
nlp = pipeline("multitask-qa-qg", model="valhalla/t5-base-qa-qg-hl")

In [ ]:
#convert to sentences from temp paragraph2, and get the first sentence, then use that first sentence to create a Q2 and A2 which will be used as a choice for user
sentences = sent_tokenize(output_string_temp2)
first_sentence = sentences[num_sentences_first]
option2 = nlp(first_sentence)[0]['question']
answer2 = nlp(first_sentence)[0]['answer']

In [ ]:
print(output_string1)
print("\n")
print("What do you think of next? Choose one of the two thoughts that cross your mind and answer in first person: \n")
print("Q1: ",option1)
print("Q2: ",option2)
print("\n")

#take user input
ans = input("Your Answer: ")

In [ ]:
#some datapreprocessing on the answer from user
ans = ans.replace("I", "you")
ans = ans.replace("my", "your")
ans = ans.replace("me", "you")
if ans[0].islower():
    ans = ans.capitalize()

In [ ]:
#We concatenate Paragraph1 with user's Answer1 to form Paragraph2. This will now be fed into the GPT again - which will generate 2 hidden paras to create Q3 and Q4 and and so loop continues
output_string2 = output_string1 + " " + ans + "."
print(output_string2)

In [ ]:
#storing number of words in Paragraph2
count = len(output_string2.split())

#send the concatenated string to gen - and thus make Paragraph2
output_string_new = generate_story_larger(output_string2,count)
blob = TextBlob(output_string_new)
newscore = blob.sentiment.polarity
output_string2 = output_string_new
output_string2 = output_string2[:output_string2.rfind('.')]
output_string2=output_string2+"."

#Paragraph2
print(output_string2)

NEW LOOP

In [ ]:
#storing number of words in Paragraph2
count = len(output_string2.split())

#generate two paragraphs from Paragraph2, both will be hidden from user. Both will be used to generate user choice options later
initialscore = newscore
newscore = 1000
while(newscore>=(initialscore)): #keep generating until sentiment/drama score is lower than paragraph1's score
    output_string_temp3 = generate_story_larger(output_string2,count)
    blob = TextBlob(output_string_temp3)
    newscore = blob.sentiment.polarity

# Print the temp paragraph 3 sentiment score
print("Temp paragraph 3 Sentiment score:", newscore) 

output_string_temp3 = output_string_temp3[:output_string_temp3.rfind('.')]
output_string_temp3=output_string_temp3+"."
print(output_string_temp3)

print("-------------------")

newscore = 1000
while(newscore>=(initialscore)):
    output_string_temp4 = generate_story_larger(output_string2,count)
    blob = TextBlob(output_string_temp4)
    newscore = blob.sentiment.polarity

# Print the temp paragraph 4 sentiment score
print("Temp paragraph 4 Sentiment score:", newscore)

output_string_temp4 = output_string_temp4[:output_string_temp4.rfind('.')]
output_string_temp4=output_string_temp4+"."
print(output_string_temp4)

In [ ]:
nlp = pipeline("question-generation")

#number of sentences in Paragraph2,so we know when the newly generated temp paragraphs start
num_sentences_first = len(sent_tokenize(output_string2))

#convert to sentences from temp paragraph3, and get the first sentence, then use that first sentence to create a Q3 and A3 which will be used as a choice for user
sentences = sent_tokenize(output_string_temp3)
first_sentence = sentences[num_sentences_first]
option3 = nlp(first_sentence)[0]['question']
answer3 = nlp(first_sentence)[0]['answer']

#changing up the nlp pipeline
nlp = pipeline("multitask-qa-qg", model="valhalla/t5-base-qa-qg-hl")

#convert to sentences from temp paragraph4, and get the first sentence, then use that first sentence to create a Q4 and A4 which will be used as a choice for user
sentences = sent_tokenize(output_string_temp4)
first_sentence = sentences[num_sentences_first]
option4 = nlp(first_sentence)[0]['question']
answer4 = nlp(first_sentence)[0]['answer']

print(output_string2)
print("What do you think of next? Choose one of the two thoughts that cross your mind and answer in first person: \n")
print("Q1: ",option3)
print("Q2: ",option4)
print("\n")

#take user input
ans = input("Your Answer: ")

#some datapreprocessing on the answer from user
ans = ans.replace("I", "you")
ans = ans.replace("my", "your")
ans = ans.replace("me", "you")
if ans[0].islower():
    ans = ans.capitalize()

#We concatenate Paragraph2 with user's Answer2 to form Paragraph3. This will now be fed into the GPT again.
output_string3 = output_string2 + " " + ans + "."


#storing number of words in Paragraph3
count = len(output_string3.split())
#send the concatenated string to gen - and thus make Paragraph3
initialscore = newscore
newscore = 1000
# while(newscore>=(initialscore)): #keep generating until sentiment/drama score is lower than paragraph1's score
output_string_new = generate_story_larger(output_string2,count)
blob = TextBlob(output_string_new)
newscore = blob.sentiment.polarity
output_string3 = output_string_new
output_string3 = output_string3[:output_string3.rfind('.')]
output_string3=output_string2+"."

print(output_string3)

NEW LOOP

In [ ]:
#storing number of words in Paragraph3
count = len(output_string3.split())

#generate two paragraphs from Paragraph3, both will be hidden from user. Both will be used to generate user choice options later
initialscore = newscore
newscore = 0.01
while(newscore<=(initialscore)): #keep generating until sentiment/drama score is higher than paragraph1's score
    output_string_temp5 = generate_story_larger(output_string3,count)
    blob = TextBlob(output_string_temp5)
    newscore = blob.sentiment.polarity

# Print the temp paragraph 5 sentiment score
print("Temp paragraph 5 Sentiment score:", newscore)    

output_string_temp5 = output_string_temp5[:output_string_temp5.rfind('.')]
output_string_temp5=output_string_temp5+"."
print(output_string_temp5)

print("---------------------------------")

newscore = 0.01
while(newscore<=(initialscore)):
    output_string_temp6 = generate_story_larger(output_string3,count)
    blob = TextBlob(output_string_temp6)
    newscore = blob.sentiment.polarity

# Print the temp paragraph 6 sentiment score
print("Temp paragraph 6 Sentiment score:", newscore)

output_string_temp6 = output_string_temp6[:output_string_temp6.rfind('.')]
output_string_temp6=output_string_temp6+"."
print(output_string_temp6)

In [ ]:
nlp = pipeline("question-generation")

#number of sentences in Paragraph3,so we know when the newly generated temp paragraphs start
num_sentences_first = len(sent_tokenize(output_string3))

#convert to sentences from temp paragraph3, and get the first sentence, then use that first sentence to create a Q3 and A3 which will be used as a choice for user
sentences = sent_tokenize(output_string_temp5)
first_sentence = sentences[num_sentences_first]
option5 = nlp(first_sentence)[0]['question']
answer5 = nlp(first_sentence)[0]['answer']

#changing up the nlp pipeline
nlp = pipeline("multitask-qa-qg", model="valhalla/t5-base-qa-qg-hl")

#convert to sentences from temp paragraph4, and get the first sentence, then use that first sentence to create a Q4 and A4 which will be used as a choice for user
sentences = sent_tokenize(output_string_temp6)
first_sentence = sentences[num_sentences_first]
option6 = nlp(first_sentence)[0]['question']
answer6 = nlp(first_sentence)[0]['answer']

print("What do you think of next? Choose one of the two thoughts that cross your mind and answer in first person: \n")
print("Q1: ",option5)
print("Q2: ",option6)
print("\n")

#take user input
ans = input("Your Answer: ")

#some datapreprocessing on the answer from user
ans = ans.replace("I", "you")
ans = ans.replace("my", "your")
ans = ans.replace("me", "you")
if ans[0].islower():
    ans = ans.capitalize()

#We concatenate Paragraph2 with user's Answer2 to form Paragraph3. This will now be fed into the GPT again
output_string4 = output_string3 + " " + ans + "."

Final push into GPT to generate the last paragraph of story after 3 rounds of User inputs

In [ ]:
#storing number of words in Paragraph4
count = len(output_string4.split())

#generate final Paragraph using Paragraph 4 as input
output_string_temp7 = generate_story_larger(output_string4,count)
blob = TextBlob(output_string_temp7)
newscore = blob.sentiment.polarity
 

output_string_temp7 = output_string_temp7[:output_string_temp7.rfind('.')]
output_string_temp7=output_string_temp7+"."
print(output_string_temp7)

print("\n")
print("-------------------------------------------------------")
print("\n")
print("PAY $60 TO UNLOCK THE REST OF THE GAME.")